# Code optimisation and profiling with `cProfile`

## There are many routes to faster more efficient code
*  Good coding practice (comes with experience, reading, Google searches, the PEPS, ...)
*  Using timers (`time` module, %timeit `ipython` magic)
*  Using profilers to find the bottlenecks
*  Going outside of Python with `f2py` and `cython`
    *  `f2py`: calling Fortran from Python
    *  `cython`: calling C from Python
    *  Easy to implement, but not covered here


# 1. Tip for `numpy`

## Use object methods where possible

### In the terminal, open ipython

In [3]:
import numpy as np

In [7]:
# Generate a multidimensional array
x = np.arange(500.).reshape(10, 5, -1)
np.random.shuffle(x)
print x.shape


(10, 5, 10)


### ipython %magic functions

In [10]:
%

ERROR: Line magic function `%` not found.


### We will do a reduction operation on `x` and use the ipython %magic function `%timeit` to time it

In [11]:
%timeit np.sum(x) # Here we use the numpy attribute *sum*

The slowest run took 10.85 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 5.45 µs per loop


In [12]:
%timeit x.sum() # Here we use the object method *sum*

The slowest run took 9.48 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 4.33 µs per loop


### Why the difference?
*  numpy methods have overheads (type?, shape?)
___

## In-place operations

In [13]:
x = np.arange(100000.)

In [14]:
%%timeit y = np.zeros((1000, 100))
for i in x:
    y = y + i

1 loops, best of 3: 5.28 s per loop


In [15]:
%%timeit y = np.zeros((1000, 100))
for i in x:
    y += i

1 loops, best of 3: 4.33 s per loop


___
# 2. cProfile

A builtin Python code profiler
* For larger codes where we don't have time to use `%timeit` on every line

Code below calculates the Julia set (see https://en.wikipedia.org/wiki/Julia_set), taken from https://github.com/mynameisfiber/high_performance_python

* This book, "High Performance Python" by Gorelick & Ozsvald, is highly recommended
* See also http://ianozsvald.com/HighPerformancePythonfromTrainingatEuroPython2011_v0.2.pdf

In [16]:
"""Julia set generator with optional Pillow-based image drawing"""
from PIL import Image
import array

def show_greyscale(output_raw, width, height, max_iterations):
    """Convert list to array, show using PIL"""
    # convert our output to PIL-compatible input
    # scale to [0...255]
    max_iterations = float(max(output_raw))
    print max_iterations
    scale_factor = float(max_iterations)
    scaled = [int(o / scale_factor * 255) for o in output_raw]
    output = array.array('B', scaled)  # array of unsigned ints
    # display with PIL
    im = Image.new("L", (width, width))
    # EXPLAIN RAW L 0 -1
    im.frombytes(output.tostring(), "raw", "L", 0, -1)
    im.show()


def show_false_greyscale(output_raw, width, height, max_iterations):
    """Convert list to array, show using PIL"""
    # convert our output to PIL-compatible input
    # sanity check our 1D array and desired 2D form
    assert width * height == len(output_raw)
    # rescale output_raw to be in the inclusive range [0..255]
    max_value = float(max(output_raw))
    output_raw_limited = [int(float(o) / max_value * 255) for o in output_raw]
    # create a slightly fancy colour map that shows colour changes with
    # increased contrast (thanks to John Montgomery)
    output_rgb = (
        (o + (256 * o) + (256 ** 2) * o) * 16 for o in output_raw_limited)  # fancier
    # array of unsigned ints (size is platform specific)
    output_rgb = array.array('I', output_rgb)
    # display with PIL/pillow
    im = Image.new("RGB", (width, height))
    # EXPLAIN RGBX L 0 -1
    im.frombytes(output_rgb.tostring(), "raw", "RGBX", 0, -1)
    im.show()




In [17]:
import time

# area of complex space to investigate
x1, x2, y1, y2 = -1.8, 1.8, -1.8, 1.8
c_real, c_imag = -0.62772, -.42193

def calculate_z_serial_purepython(maxiter, zs, cs):
    """Calculate output list using Julia update rule"""
    output = [0] * len(zs)
    for i in range(len(zs)):
        n = 0
        z = zs[i]
        c = cs[i]
        while abs(z) < 2 and n < maxiter:
            z = z * z + c
            n += 1
        output[i] = n
    return output


def calc_pure_python(draw_output, desired_width, max_iterations):
    """Create a list of complex co-ordinates (zs) and complex parameters (cs), build Julia set and display"""
    x_step = (float(x2 - x1) / float(desired_width))
    y_step = (float(y1 - y2) / float(desired_width))
    x = []
    y = []
    ycoord = y2
    while ycoord > y1:
        y.append(ycoord)
        ycoord += y_step
    xcoord = x1
    while xcoord < x2:
        x.append(xcoord)
        xcoord += x_step
    # set width and height to the generated pixel counts, rather than the
    # pre-rounding desired width and height
    width = len(x)
    height = len(y)
    # build a list of co-ordinates and the initial condition for each cell.
    # Note that our initial condition is a constant and could easily be removed,
    # we use it to simulate a real-world scenario with several inputs to our
    # function
    zs = []
    cs = []
    for ycoord in y:
        for xcoord in x:
            zs.append(complex(xcoord, ycoord))
            cs.append(complex(c_real, c_imag))

    print "Length of x:", len(x)
    print "Total elements:", len(zs)
    start_time = time.time()
    output = calculate_z_serial_purepython(max_iterations, zs, cs)
    end_time = time.time()
    secs = end_time - start_time
    print calculate_z_serial_purepython.func_name + " took", secs, "seconds"

    # this sum is expected for 1000^2 grid with 300 iterations
    assert sum(output) == 33219980

    if draw_output:
        #show_false_greyscale(output, width, height, max_iterations)
        show_greyscale(output, width, height, max_iterations)


if __name__ == "__main__":
    # Calculate the Julia set using a pure Python solution with
    # reasonable defaults for a laptop
    # set draw_output to True to use PIL to draw an image
    calc_pure_python(draw_output=True, desired_width=1000, max_iterations=300)

Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 7.08579397202 seconds
300.0


## How well is this script performing and can we improve it?



* We must use `cProfile`

Go to https://github.com/mynameisfiber/high_performance_python/tree/master/01_profiling/cpu_profiling and download **julia1_nopil.py**

At the terminal, run:

    $ python -m cProfile julia1_nopil.py

### What if we want to sort by cumulative time?

    $ python -m cProfile -s cumtime julia1_nopil.py

___
### Another way to use cProfile: `cprofilev`

https://ymichael.com/2014/03/08/profiling-python-with-cprofile.html

https://github.com/ymichael/cprofilev

#### Install cprofilev
$ pip install cprofilev

#### Run the code, but now save the cProfile output
$ python -m cProfile -o profile_output julia1_nopil.py

#### Call it with your cprofile output
$ cprofilev /path/to/cprofile/output

#### Navigate to http://localhost:4000 and you can see and manipulate the results in the web browser

___
# Exercise (30 minutes)
The methods `calculate_z_serial_purepython` and `calc_pure_python` above are written with *pure Python*; this means that they don't use `numpy`.

Work alone or with a partner.

Starting with `calc_pure_python` try to introduce `numpy` into the code, and use `cProfile` and `cprofilev` to evaluate any changes in performance
*  You will need to find equivalent *numpy* methods
*  Make just one or two small changes, then test them, etc...

___
# Finally, the PEPS

Yesterday, you learned about `PEP20` (the Zen of Python).  Today we have `PEP8`, the style guide for Python coding:

https://www.python.org/dev/peps/pep-0008/

Many editors (e.g., PyCharm, Atom, ...) have functions that will automatically format your code according to `PEP8`